# Resize images

In [ ]:
from pathlib import Path
from PIL import Image, ImageOps
import sys
import time

SUPPORTED_EXTENSIONS = {".jpg", ".jpeg"}


def is_animated_gif(image: Image.Image) -> bool:
    return getattr(image, "is_animated", False) and getattr(image, "n_frames", 1) > 1


def compute_target_size(width: int, height: int) -> tuple[int, int]:
    new_width = max(1, width // 4)
    new_height = max(1, height // 4)
    return new_width, new_height


def process_image(path: Path) -> tuple[int, int]:
    """
    Resize an image in-place by 4x reduction while keeping aspect ratio.
    Returns a tuple of (before_bytes, after_bytes). Returns (0, 0) when skipped.
    """
    before_bytes = path.stat().st_size

    with Image.open(path) as img:
        img_format = (img.format or "").upper()
        img = ImageOps.exif_transpose(img)

        if img_format == "GIF" and is_animated_gif(img):
            # Skip animated GIFs to avoid breaking animation
            return (0, 0)

        width, height = img.size
        target_width, target_height = compute_target_size(width, height)

        if (target_width, target_height) == (width, height):
            # Nothing to do
            return (0, 0)

        resized = img.resize((target_width, target_height), resample=Image.Resampling.LANCZOS)

        save_kwargs = {}
        suffix = path.suffix.lower()

        if img_format in ("JPEG", "JPG") or suffix in (".jpg", ".jpeg"):
            save_kwargs.update(dict(format="JPEG", quality=85, optimize=True, progressive=True, subsampling=2))
            exif_bytes = img.info.get("exif")
            if exif_bytes:
                save_kwargs["exif"] = exif_bytes
        elif img_format == "PNG" or suffix == ".png":
            save_kwargs.update(dict(format="PNG", optimize=True))
        elif img_format == "WEBP" or suffix == ".webp":
            save_kwargs.update(dict(format="WEBP", quality=85, method=6))
        elif img_format == "GIF" or suffix == ".gif":
            save_kwargs.update(dict(format="GIF"))
        # else: rely on Pillow inferring format from extension

        resized.save(path, **save_kwargs)

    after_bytes = path.stat().st_size
    return before_bytes, after_bytes


def main():
    repo_root = Path.cwd()
    img_root = repo_root / "img"

    if not img_root.exists():
        print(f"Image directory not found: {img_root}")
        return

    total_before = 0
    total_after = 0
    processed = 0
    skipped = 0
    errors = 0

    start_time = time.time()

    for file_path in img_root.rglob("*"):
        if not file_path.is_file():
            continue
        if file_path.suffix.lower() not in SUPPORTED_EXTENSIONS:
            continue
        try:
            before, after = process_image(file_path)
            if before == 0 and after == 0:
                skipped += 1
                continue
            processed += 1
            total_before += before
            total_after += after
        except Exception as exc:
            errors += 1
            print(f"Failed to process {file_path}: {exc}", file=sys.stderr)

    elapsed = time.time() - start_time
    saved = total_before - total_after
    pct = (saved / total_before * 100.0) if total_before else 0.0

    print(f"Processed: {processed}, Skipped: {skipped}, Errors: {errors}")
    print(f"Bytes before: {total_before:,} after: {total_after:,} saved: {saved:,} ({pct:.1f}%)")
    print(f"Elapsed: {elapsed:.1f}s")


if __name__ == "__main__":
    main()

